In [1]:
import arviz as az
import numpy as np
import pandas as pd
import stan
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import nest_asyncio

In [4]:
import sys
sys.path.append('../madule')
import utils

ModuleNotFoundError: No module named 'utils'

In [9]:
# Number of chains in MCMC procedure
nChains = 2
# The number of iteration or samples for each chain in MCM procedure
nSamples=10 

dataAll = pd.read_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/First-level-analysis/behAll.csv')

# data from group label 2 particiapnt
dataGroup2 = dataAll.loc[dataAll['group']==2]   


In [3]:
np.unique(dataGroup2['sub_ID'])

array(['sub-012', 'sub-026', 'sub-030', 'sub-033', 'sub-034', 'sub-036',
       'sub-047', 'sub-048', 'sub-054', 'sub-060', 'sub-064', 'sub-067',
       'sub-069', 'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079',
       'sub-080', 'sub-081', 'sub-083', 'sub-088', 'sub-090'],
      dtype=object)

In [4]:
dataGroup2 = dataGroup2.loc[(dataGroup2['sub_ID'] == 'sub-012')|
                            (dataGroup2['sub_ID'] == 'sub-026')|
                            (dataGroup2['sub_ID'] == 'sub-030')|
                            (dataGroup2['sub_ID'] == 'sub-034')|
                            (dataGroup2['sub_ID'] == 'sub-036')]

In [11]:
nCond = 2 
nSes = 2 
# number of participant
nParts = len(np.unique(dataGroup2.sub_ID))
# participant indeces
participant = dataGroup2.sub_ID.replace(np.unique(dataGroup2.sub_ID),
                      np.arange(1, nParts +1, 1))
# condition indeces
condition = dataGroup2.block.replace('Act',1).replace('Stim',2)


# Put required data for stan model
dataStan = {'N':int(dataGroup2.shape[0]),  
            'nParts': nParts,
            'nCond':2, 
            'nSes':2, 
            'pushed':np.array(dataGroup2.pushed).astype(int),  
            'yellowChosen':np.array(dataGroup2.yellowChosen).astype(int), 
            'winAmtPushable':np.array(dataGroup2.winAmtPushable).astype(int), 
            'winAmtYellow':np.array(dataGroup2.winAmtYellow).astype(int), 
            'rewarded':np.array(dataGroup2.correctChoice).astype(int),  
            'p_push_init':.5, 
            'p_yell_init':.5,        
            'participant':np.array(participant).astype(int),      
            'session':np.array(dataGroup2.session).astype(int),
            'condition':np.array(condition).astype(int)}

In [12]:
# Loading the RL Stan Model
file_name = 'stan_models/RL_hier_3.stan' 
file_read = open(file_name, 'r')
stan_model = file_read.read()
# Use nest-asyncio.This package is needed because Jupter Notebook blocks the use of certain asyncio functions
nest_asyncio.apply()
# Building Stan Model realted to our proposed model
posterior = stan.build(stan_model, data = dataStan)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_llwvkor7/model_z3ueowh7.stan', line 10, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_llwvkor7/model_z3ueowh7.stan', line 11, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_llwvkor7/model_z3ueowh7.stan', line 12, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-for

Future exception was never retrieved
future: <Future finished exception=RuntimeError('write: Broken pipe [system:32]')>
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/mrhome/amingk/anaconda3/lib/python3.10/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/mrhome/amingk/anaconda3/lib/python3.10/site-packages/httpstan/services_stub.py", line 47, in _make_lazy_function_wrapper_helper
    return function(*args, **kwargs)  # type: ignore
RuntimeError: write: Broken pipe [system:32]
"""

The above exception was the direct cause of the following exception:

RuntimeError: write: Broken pipe [system:32]


In [13]:
initials = [] # initial sampling
for c in range(0, nChains):
    chaininit = {
        'alphaAct_sd': np.random.uniform(.05, .2),
        'alphaClr_sd': np.random.uniform(.05, .2),        
        'weightAct_sd': np.random.uniform(.05, .2),
        'sensitivity_sd': np.random.uniform(.05, .2),
        'alphaAct_hier': np.random.uniform(.3, .7, size=(nSes, nCond)),
        'alphaClr_hier': np.random.uniform(.3, .7, size=(nSes, nCond)),
        'weightAct_hier': np.random.uniform(.3, .7, size=(nSes, nCond)),        
        'sensitivity_hier': np.random.uniform(.1, .1, size=(nSes)),
        'alphaAct': np.random.uniform(.1, .9, size=(nParts, nSes, nCond)),       
        'alphaClr': np.random.uniform(.1, .9, size=(nParts, nSes, nCond)),
        'weightAct': np.random.uniform(.1, .9, size=(nParts, nSes, nCond)),   
        'sensitivity': np.random.uniform(.1, .2, size=(nParts, nSes))
    }
    initials.append(chaininit) 

In [14]:
# Start for taking samples from parameters in the Stan Model
fit = posterior.sample(num_chains=nChains, num_samples=nSamples, init = initials)

Sampling:   0%
Sampling:   0% (1/2020)
Sampling:   0% (2/2020)


In [69]:
subName ='sub'

# Extracting posterior distributions for each of four main unkhown parameters
alphaAct_ = fit["alphaAct"][0,:] 
alphaClr_ = fit["alphaClr"][0,:]  
weightAct_ = fit["weightAct"][0,:]  
beta_ = fit["sensitivity"][0,:]  

# Figure of model fit results in two column and two rows
fig = plt.figure(figsize=(10, 6), tight_layout=True)
rows = 2
columns = 2

# Weghtening
fig.add_subplot(rows, columns, 1)
sns.histplot(weightAct_[0, 0], kde=True, stat='density')
sns.histplot(weightAct_[0, 1], kde=True, stat='density')
sns.histplot(weightAct_[1, 0], kde=True, stat='density')
sns.histplot(weightAct_[1, 1], kde=True, stat='density')
plt.title(subName + ', Weightening', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.xlabel('$w_{(A)}$', fontsize=14)
plt.legend(['Ses 1, Act', 'Ses 1, Clr', 'Ses 2, Act', 'Ses 2, Clr'], fontsize=8)


# Sensitivity
fig.add_subplot(rows, columns, 2)
sns.histplot(beta_[0], kde=True, stat='density')
sns.histplot(beta_[1], kde=True, stat='density')
plt.title(subName + ', Sensitivity', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.xlabel(r'$\beta$', fontsize=14)
plt.legend(['Ses 1', 'Ses 2'], fontsize=8)


# Action Learning Rate
fig.add_subplot(rows, columns, 3)
sns.histplot(alphaAct_[0, 0], kde=True, stat='density')
sns.histplot(alphaAct_[0, 1], kde=True, stat='density')
sns.histplot(alphaAct_[1, 0], kde=True, stat='density')
sns.histplot(alphaAct_[1, 1], kde=True, stat='density')
plt.title(subName + ', Action Learning Rate', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.xlabel(r'$ \alpha_{(A)} $', fontsize=14)
plt.legend(['Ses 1, Act', 'Ses 1, Clr', 'Ses 2, Act', 'Ses 2, Clr'], fontsize=8)

# Color Learning Rate
fig.add_subplot(rows, columns, 4)
sns.histplot(alphaClr_[0, 0], kde=True, stat='density')
sns.histplot(alphaClr_[0, 1], kde=True, stat='density')
sns.histplot(alphaClr_[1, 0], kde=True, stat='density')
sns.histplot(alphaClr_[1, 1], kde=True, stat='density')
plt.title(subName + ', Color Learning Rate', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.xlabel(r'$ \alpha_{(C)} $', fontsize=14)
plt.legend(['Ses 1, Act', 'Ses 1, Clr', 'Ses 2, Act', 'Ses 2, Clr'], fontsize=8)

plt.subplots_adjust(wspace=10.)

#fig.savefig(subMainDirec + subName + '/' + subName +'_inv_rlActClr.png', dpi=300)

TypeError: 'NoneType' object is not subscriptable

In [103]:
az.summary(fit)

/mrhome/amingk/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:180: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/mrhome/amingk/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:180: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/mrhome/amingk/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:213: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/mrhome/amingk/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:213: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/mrhome/amingk/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
arviz - WARNING - Shape validation failed: input_shape: (1, 100), 

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"alphaAct_aHier[0, 0]",0.004,0.000,0.004,0.004,0.000,0.000,7.0,17.0,NaN
"alphaAct_aHier[0, 1]",0.008,0.000,0.007,0.008,0.000,0.000,2.0,8.0,NaN
"alphaAct_aHier[1, 0]",0.010,0.001,0.008,0.012,0.001,0.001,1.0,11.0,NaN
"alphaAct_aHier[1, 1]",0.005,0.000,0.004,0.005,0.000,0.000,2.0,16.0,NaN
"alphaClr_aHier[0, 0]",0.888,0.192,0.594,1.256,0.151,0.135,1.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...
soft_max_EV[314],0.500,0.000,0.500,0.500,0.000,0.000,100.0,100.0,NaN
soft_max_EV[315],0.500,0.000,0.500,0.500,0.000,0.000,100.0,100.0,NaN
soft_max_EV[316],0.500,0.000,0.500,0.500,0.000,0.000,100.0,100.0,NaN
soft_max_EV[317],0.500,0.000,0.500,0.500,0.000,0.000,100.0,100.0,NaN
